<a href="https://colab.research.google.com/github/tomasonjo/blogs/blob/master/llm/llama_relik.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Clone the repository
!git clone https://github.com/richardpaulhudson/coreferee

# Change to the repository directory
%cd coreferee

# Modify the setup.cfg file
!sed -i '39s/spacy>=3.0.0,<3.6.0/spacy>=3.0.0,<3.8.0/' setup.cfg
!sed -i 's/to_version:             3.3.0/to_version:             3.8.0/g' coreferee/lang/en/config.cfg
# Verify the change
!cat setup.cfg | grep spacy

# Install the package
!pip install .

Cloning into 'coreferee'...
remote: Enumerating objects: 424, done.
remote: Counting objects: 100% (99/99), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 424 (delta 48), reused 60 (delta 37), pack-reused 325
Receiving objects: 100% (424/424), 248.60 MiB | 25.30 MiB/s, done.
Resolving deltas: 100% (173/173), done.
Updating files: 100% (153/153), done.
/content/coreferee
keywords= nlp, spacy, spacy-extension, python, machine-learning, coreference, anaphora, coreference-resolution, anaphora-resolution
  spacy>=3.0.0,<3.8.0
spacy_factories =
Processing /content/coreferee
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for coreferee: filename=coreferee-1.4.1-py3-none-any.whl size=182569 sha256=71834b7e7dfefee0ae2a0d433b41c1ce954a2377f78f7c8cba496cab42718287
  Stored in directory: /tmp/pip-ephem-wheel-cache-m2s51cu_/wheels/a3/71/a1/572217811ada49a1a01439dd1902bf93f82f

In [11]:
!pip install --quiet llama-index-extractors-relik llama-index-graph-stores-neo4j llama-index-llms-openai llama-index

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 52.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 20.8 MB/s eta 0:00:00


In [3]:
!python -m spacy download en_core_web_lg
!python -m coreferee install en

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 2.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
Processing ./models/en
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for coreferee-model-en: filename=coreferee_model_en-1.0.0-py3-none-any.whl size=65422507 sha256=c0d69b333a6f0e5147bd386302195956b1ca1bc7ab5fc6bb3d498cd6972b94c4
  Stored in directory: /tmp/pip-ephem-wheel-cache-q03wo5x9/wheels/f6/be/ad/ab1b76b72291d45f7a92a9a29098c985a2fbc9ad9f97c658ac
Successfully built coreferee-model-en


In [14]:
import nest_asyncio

nest_asyncio.apply()

In [4]:
import spacy, coreferee

coref_nlp = spacy.load('en_core_web_lg')
coref_nlp.add_pipe('coreferee')


def coref_text(text):
    coref_doc = coref_nlp(text)
    resolved_text = ""

    for token in coref_doc:
        repres = coref_doc._.coref_chains.resolve(token)
        if repres:
            resolved_text += " " + " and ".join(
                [
                    t.text
                    if t.ent_type_ == ""
                    else [e.text for e in coref_doc.ents if t in e][0]
                    for t in repres
                ]
            )
        else:
            resolved_text += " " + token.text

    return resolved_text

print(coref_text("Tomaz is so cool. He can solve various Python dependencies and not cry"))

 Tomaz is so cool . Tomaz can solve various Python dependencies and not cry


In [16]:
import pandas as pd
from llama_index.core import Document
from datetime import datetime

NUMBER_OF_ARTICLES = 10

print(datetime.now())

news = pd.read_csv("https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/news_articles.csv")
news = news.head(NUMBER_OF_ARTICLES)
news['coref_text'] = news['text'].apply(coref_text)
documents = [Document(text=f"{row['title']}: {row['coref_text']}") for i, row in news.iterrows()]
print(datetime.now())

print(coref_text(news['text'][5]))

2024-08-06 17:09:58.508024
2024-08-06 17:10:02.596685
 Jude Bellingham has also insisted Jude Bellingham move to Real Madrid had nothing to do with money , as Jude Bellingham explained how Jude Bellingham transfer was wrapped up so quickly . 
 “ Money is not a thing for me , ” Jude Bellingham said . 
 “ I do nt think about money at all when I make these kinds of decisions . I never have and I never will . I play the game purely out of love . 
 “ I spoke with people from Real Madrid when I was given permission by Borussia Dortmund and I love the feeling I got from the club . I could n’t hide feeling . I told people straight away what I felt and after that happened on Monday feeling all happened quickly . ” 
 Monday also opened up on how love was given the No.5 shirt , which was worn by defender Jesus Vallejo last season , and what shirt means to Jesus Vallejo to have Zidane ’s old number on Zidane back . 
 “ For a start I ’d like to thank Jesus Jesus Vallejo for letting me wear the No.5

In [19]:
from llama_index.graph_stores.neo4j import Neo4jPGStore

username="neo4j"
password="rubber-cuffs-radiator"
url="bolt://54.89.19.156:7687"

graph_store = Neo4jPGStore(
    username=username,
    password=password,
    url=url,
    refresh_schema=False
)

In [20]:
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI

import os
os.environ["OPENAI_API_KEY"] = "sk-"

llm = OpenAI(model="gpt-4o", temperature=0.0)
embed_model = OpenAIEmbedding(model_name="text-embedding-3-small")

In [21]:
from llama_index.extractors.relik.base import RelikPathExtractor
from llama_index.core import PropertyGraphIndex

# Use relik-ie/relik-cie-small if possible
# Run on cuda if possible
relik = RelikPathExtractor() # device="cuda"

index = PropertyGraphIndex.from_documents(
    documents,
    kg_extractors=[relik],
    llm=llm,
    embed_model=embed_model,
    property_graph_store=graph_store,
    show_progress=True,
)

                ___              __         
               /\_ \      __    /\ \        
 _ __     __   \//\ \    /\_\   \ \ \/'\    
/\`'__\ /'__`\   \ \ \   \/\ \   \ \ , <    
\ \ \/ /\  __/    \_\ \_  \ \ \   \ \ \\`\  
 \ \_\ \ \____\   /\____\  \ \_\   \ \_\ \_\
  \/_/  \/____/   \/____/   \/_/    \/_/\/_/
                                            
                                            



/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


[2024-08-06 17:12:53,037] [WARNING] [relik.common.utils.download_from_hf:342] [PID:2861] [RANK:0] Couldn't download index.faiss from relik-ie/encoder-e5-small-v2-wikipedia-relations-index, ignoring


Parsing nodes:   0%|          | 0/10 [00:00<?, ?it/s]

Extracting triples:  30%|███       | 3/10 [00:05<00:15,  2.19s/it]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_fast.py:311: RuntimeWarning: coroutine 'arun_transformations' was never awaited
  encoding_dict["attention_mask"].append(e.attention_mask)
Generating embeddings: 100%|██████████| 5/5 [00:01<00:00,  3.62it/s]


In [22]:
query_engine = index.as_query_engine(include_text=True)

response = query_engine.query("What happened at Ryanair?")

print(str(response))

Ryanair sacked its chief pilot, Aidan Murray, following an investigation into alleged sexual harassment of female colleagues. The airline stated that the chief pilot was dismissed due to a pattern of inappropriate behavior towards several female pilots. The chief people officer, Darrell Hughes, confirmed the termination of Mr. Murray's employment, citing a breach of the anti-harassment policy. The dismissal came after reports of Mr. Murray harassing junior colleagues and altering flight rosters to fly with specific female pilots. The airline emphasized the importance of a safe and secure work environment for its staff.
